# HeudiConv Basics: DICOM to BIDS Conversion

This notebook demonstrates how to use the HeudiConv runner to convert DICOM files to BIDS format.

## Overview

HeudiConv is a flexible DICOM converter that creates BIDS-compliant datasets. It uses heuristic files to map DICOM series to BIDS-compliant names.

## Prerequisites

- Docker installed and running
- DICOM data organized by participant
- A heuristic file defining the conversion rules

## Setup

In [1]:
from pathlib import Path

from voxelops import (
    HeudiconvDefaults,
    HeudiconvInputs,
    run_heudiconv,
)

## Define Paths

Update these paths to match your data structure:

In [ ]:
# Input paths -- update these to match your data
dicom_dir = Path("/data/raw/dicom/")
participant = "01"
heuristic_file = Path("/config/heuristics/brain_bank.py")

# Output paths (optional -- will use defaults if not specified)
output_dir = Path("/data/bids/")

## Basic Usage

### Option 1: Use Default Configuration

In [3]:
# Create inputs
inputs = HeudiconvInputs(
    dicom_dir=dicom_dir,
    participant=participant,
    session="01",
    output_dir=output_dir,
    # work_dir=work_dir,
)

In [ ]:
# Run with defaults
result = run_heudiconv(inputs, heuristic=heuristic_file, overwrite=True)

# Check result
print(f"Success: {result['success']}")
print(f"Duration: {result['duration_human']}")
print(f"Output: {result['expected_outputs'].bids_dir}")

In [ ]:
# Run with parameter overrides
result = run_heudiconv(
    inputs,
    overwrite=True,  # Overwrite existing output
    bids_validate=True,  # Run BIDS validator
    docker_image="nipy/heudiconv:0.13.1",  # Use specific version
)

print(f"Success: {result['success']}")
print(f"Exit code: {result['exit_code']}")

### Option 3: Use Custom Configuration

In [ ]:
# Run with parameter overrides
result = run_heudiconv(
    inputs,
    overwrite=True,
    bids_validator=True,  # Run BIDS validator after conversion
    docker_image="nipy/heudiconv:1.3.4",  # Pin specific version
)

print(f"Success: {result['success']}")
print(f"Exit code: {result['exit_code']}")

## Inspect Execution Record

The execution record contains complete information about the run:

In [ ]:
# Create custom configuration
config = HeudiconvDefaults(
    overwrite=True,
    bids_validator=True,
    docker_image="nipy/heudiconv:1.3.4",
)

# Run with custom config
result = run_heudiconv(inputs, config)

print(f"Success: {result['success']}")

## Check Expected Outputs

In [ ]:
outputs = result["expected_outputs"]

print("Expected outputs:")
print(f"BIDS directory: {outputs.bids_dir}")
print(f"Participant directory: {outputs.participant_dir}")
print(f"Work directory: {outputs.work_dir}")

# Check if outputs exist
print("\nOutput validation:")
print(f"BIDS dir exists: {outputs.bids_dir.exists()}")
print(f"Participant dir exists: {outputs.participant_dir.exists()}")

# List files in participant directory
if outputs.participant_dir.exists():
    print(f"\nFiles in {outputs.participant_dir}:")
    for f in outputs.participant_dir.rglob("*"):
        if f.is_file():
            print(f"  {f.relative_to(outputs.participant_dir)}")

## Save Execution Record

Save the execution record to a database or file:

In [ ]:
outputs = result["expected_outputs"]

print("Expected outputs:")
print(f"  BIDS directory: {outputs.bids_dir}")
print(f"  Participant directory: {outputs.participant_dir}")
print(f"  Dataset description: {outputs.dataset_description}")

# Check if outputs exist
print("\nOutput validation:")
print(f"  BIDS dir exists: {outputs.bids_dir.exists()}")
print(f"  Participant dir exists: {outputs.participant_dir.exists()}")

# List files in participant directory
if outputs.participant_dir.exists():
    print(f"\nFiles in {outputs.participant_dir}:")
    for f in outputs.participant_dir.rglob("*"):
        if f.is_file():
            print(f"  {f.relative_to(outputs.participant_dir)}")

## Error Handling

In [ ]:
from voxelops.exceptions import (
    InputValidationError,
    ProcedureExecutionError,
)

try:
    result = run_heudiconv(inputs)
    print(f"Success: {result['success']}")
except InputValidationError as e:
    print(f"Input validation failed: {e}")
except ProcedureExecutionError as e:
    print(f"Execution failed: {e}")
    print(f"Check logs at: {result.get('log_file')}")
except Exception as e:
    print(f"Unexpected error: {e}")

## Batch Processing Multiple Participants

In [ ]:
# List of participants to process
participants = ["01", "02", "03", "04", "05"]

results = []

for participant in participants:
    print(f"\nProcessing participant {participant}...")

    inputs = HeudiconvInputs(
        dicom_dir=dicom_dir,
        participant=participant,
        heuristic=heuristic_file,
        output_dir=output_dir,
    )

    try:
        result = run_heudiconv(inputs)
        results.append(result)
        print(f"  ✓ Success in {result['duration_human']}")
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        results.append({"participant": participant, "success": False, "error": str(e)})

# Summary
successful = sum(1 for r in results if r.get("success"))
print(
    f"\nProcessed {len(results)} participants: {successful} successful, {len(results) - successful} failed"
)

## Next Steps

After converting DICOM to BIDS:

1. Validate the BIDS dataset using the BIDS validator
2. Run quality control checks
3. Proceed to preprocessing with QSIPrep (see `02_qsiprep_basics.ipynb`)

## Tips

- **Heuristic files**: Create a reusable heuristic file for your study protocol
- **BIDS validation**: Enable `bids_validate=True` to catch errors early
- **Overwrite carefully**: Use `overwrite=True` only when necessary
- **Check logs**: If conversion fails, check the log file for detailed error messages
- **Docker image**: Pin a specific version for reproducibility